# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import pandas as pd
import requests as rq
import tqdm
import time
import os
import sqlite3 as sql

In [2]:
page = 0
limit = 10
offset = 0
id_object = 0
http_rq_list_objects = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&page={page}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
http_rq_object = 'https://наш.дом.рф/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/'

In [3]:
req = rq.get(http_rq_list_objects)

In [4]:
json_resp = req.json()

In [5]:
total = req.json().get('data').get('total')
print('Всего записей: ' + str(total))
limit = 1000
count_iter = total//limit
if total%limit > 0: count_iter = count_iter + 1
print('Количество необходимых итераций: ' + str(count_iter))

Всего записей: 10095
Количество необходимых итераций: 11


In [6]:
list_id_objects = []
for page_iter in range(count_iter):
    print('Получаем список объектов со страницы ' + str(page_iter+1), end=': ')
    offset = len(list_id_objects)
    http_rq_list_objects = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?page={page_iter}&offset={offset}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    # print('    ' + http_rq_list_objects)
    req_page_iter = rq.get(http_rq_list_objects)
    json_req_page_iter = req_page_iter.json()
    if json_req_page_iter.get('errcode') != str(0):
        print(json_req_page_iter.get('errcode'))
        break
    count_objects = len(json_req_page_iter.get('data').get('list'))
    print('    Объектов на этой странице ' + str(count_objects))
    for iter_objects in range(count_objects):
        list_id_objects.append((json_req_page_iter.get('data').get('list')[iter_objects]).get('objId'))
list_id_objects = list(set(list_id_objects))

Получаем список объектов со страницы 1:     Объектов на этой странице 1000
Получаем список объектов со страницы 2:     Объектов на этой странице 1000
Получаем список объектов со страницы 3:     Объектов на этой странице 1000
Получаем список объектов со страницы 4:     Объектов на этой странице 1000
Получаем список объектов со страницы 5:     Объектов на этой странице 1000
Получаем список объектов со страницы 6:     Объектов на этой странице 1000
Получаем список объектов со страницы 7:     Объектов на этой странице 1000
Получаем список объектов со страницы 8:     Объектов на этой странице 1000
Получаем список объектов со страницы 9:     Объектов на этой странице 1000
Получаем список объектов со страницы 10:     Объектов на этой странице 1000
Получаем список объектов со страницы 11:     Объектов на этой странице 95


In [7]:
objects_data = []
for iter_id_object in tqdm.trange(len(list_id_objects)):
    time.sleep(1/4)
    rq_obj = rq.get(http_rq_object + str(list_id_objects[iter_id_object]))
    if rq_obj.status_code != 200:
        print('Возникла ошибка с кодом [' + str(rq_obj.status_code) + ']. Описание ошибки: [' + str(rq_obj.json().get('errcode')) + ']')
    objects_data.append((rq_obj.json()).get('data'))
len(objects_data)
backup_objects_data = objects_data.copy()

 11%|█         | 1071/10093 [08:09<1:08:40,  2.19it/s]


ConnectionError: HTTPSConnectionPool(host='xn--80az8a.xn--d1aqf.xn--p1ai', port=443): Max retries exceeded with url: /%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36198 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000002557FCA1048>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))

In [11]:
objects_data[1000]

{'address': 'г Уфа, ул Армавирская, литера 2',
 'conclusion': False,
 'developer': {'buildObjCnt': 1,
  'comissObjCnt': 0,
  'companyGroupId': 7312962001,
  'devEmail': 'vesna-alt@yandex.ru',
  'devEmplMainFullNm': 'Насыров Рим Самигуллович',
  'devFactAddr': 'Респ Башкортостан город Уфа д. 1 В',
  'devFullCleanNm': 'ЖИЛОЙ КОМПЛЕКС ЮЖНЫЙ',
  'devId': 6861,
  'devInn': '0274193308',
  'devKpp': '027401001',
  'devLegalAddr': 'Башкортостан Республика, город Уфа, улица Армавирская дом 1 корпус В',
  'devOgrn': '1140280410357',
  'devOrgRegRegionCd': 2,
  'devPhoneNum': '+7(347)289-42-00',
  'devShortCleanNm': 'ЖК ЮЖНЫЙ',
  'devShortNm': 'ЖК ЮЖНЫЙ',
  'devSite': 'gk-y.info',
  'developerGroupName': 'ЖК Южный',
  'fundGuarantyFlg': 1,
  'govFundFlg': 0,
  'lastRpdId': 505802,
  'objGuarantyEscrowFlg': 0,
  'orgForm': {'fullForm': 'Общество с ограниченной ответственностью',
   'id': 4,
   'shortForm': 'ООО'},
  'problObjCnt': 0,
  'regRegionDesc': 'Башкортостан Республика'},
 'floorMax': 28,

In [14]:
set_first = set(objects_data[0].keys())
set_all = set()
for test_iter in objects_data:
    set_all = set_all ^ set(test_iter.keys())
print(set_first)
print('')
print(set_all - set_first)

{'objGreenHouseFlg', 'objProblemFlg', 'loadDttm', 'rpdNum', 'objNonlivElemCnt', 'quartography', 'objInfrstrBicycleLaneFlg', 'pdId', 'objFlatCnt', 'objectTransportInfo', 'nameObj', 'nonlivFirstFloor', 'objLivCeilingHeight', 'objLkLongitude', 'objSquareLiving', 'soldOutPerc', 'objInfrstrObjPrkngCnt', 'generalContractorNm', 'objectType', 'rpdIssueDttm', 'residentialBuildings', 'address', 'objLkLatitude', 'photoRenderDTO', 'id', 'objReady100PercDt', 'rpdPdfLink', 'objGuarantyEscrowFlg', 'objFlatSq', 'objInfrstrCurbLoweringFlg', 'objElevatorCargoCnt', 'objLkSaleDepEmail', 'objStatus', 'objLkClassDesc', 'objElemParkingCnt', 'conclusion', 'objLkFinishTypeDesc', 'objEnergyEffShortDesc', 'objInfrstrPlaygrndCnt', 'objInfrstrRampFlg', 'objElemLivingCnt', 'objCompensFundFlg', 'objTransferPlanDt', 'objLkFreePlanDesc', 'miniUrl', 'objElevatorWheelchairCnt', 'wallMaterialShortDesc', 'newBuildingId', 'isAvailableWantLiveHere', 'floorMin', 'objFloorCnt', 'objInfrstrSportGroundCnt', 'objInfrstrTrashArea

In [15]:
df_objects_data = pd.DataFrame()
df_objects_photo = pd.DataFrame()
for iter_obj in tqdm.trange(len(objects_data)):
    object_data = objects_data[iter_obj]
    df_objects_data = pd.concat([df_objects_data, pd.DataFrame(
    {'id' : object_data.get('id'),
        'address' : object_data.get('address'),
        'devEvmail' : object_data.get('developer').get('devEvmail'),
        'devFactAddr' : object_data.get('developer').get('devFactAddr'),
        'devFullCleanNm' : object_data.get('developer').get('devFullCleanNm'),
        'devInn' : object_data.get('developer').get('devInn'),
        'devKpp' : object_data.get('developer').get('devKpp'),
        'devSite' : object_data.get('developer').get('devSite'),
        'floorMax' : object_data.get('floorMax'),
        'floorMin' : object_data.get('floorMin'),
        'generalContractorNm' : object_data.get('generalContractorNm'),
        'isAvailableWantLiveHere' : object_data.get('isAvailableWantLiveHere'),
        # 'metro': object_data.get('metro', {'name':''}).get('name'),
        'metro_station': object_data.get('metro', {'name':''}).get('name'),
        'metro_line': object_data.get('metro', {'line':''}).get('line'),
        'nonlivFirstFloor' : object_data.get('nonlivFirstFloor'),
        'objElemLivingCnt' : object_data.get('objElemLivingCnt'),
        'objElemParkingCnt' : object_data.get('objElemParkingCnt'),
        'objElevatorCargoCnt' : object_data.get('objElevatorCargoCnt'),
        'objElevatorPassengerCnt' : object_data.get('objElevatorPassengerCnt'),
        'objElevatorWheelchairCnt' : object_data.get('objElevatorWheelchairCnt'),
        'objEnergyEffShortDesc' : object_data.get('objEnergyEffShortDesc'),
        'objFlatCnt' : object_data.get('objFlatCnt'),
        'objFlatSq' : object_data.get('objFlatSq'),
        'objFloorCnt' : object_data.get('objFloorCnt'),
        'objGreenHouseFlg' : object_data.get('objGreenHouseFlg'),
        'objInfrstrBicycleLaneFlg' : object_data.get('objInfrstrBicycleLaneFlg'),
        'objInfrstrCurbLoweringFlg' : object_data.get('objInfrstrCurbLoweringFlg'),
        'objInfrstrNotObjPrkngCnt' : object_data.get('objInfrstrNotObjPrkngCnt'),
        'objInfrstrObjPrkngCnt' : object_data.get('objInfrstrObjPrkngCnt'),
        'objInfrstrPlaygrndCnt' : object_data.get('objInfrstrPlaygrndCnt'),
        'objInfrstrRampFlg' : object_data.get('objInfrstrRampFlg'),
        'objInfrstrSportGroundCnt' : object_data.get('objInfrstrSportGroundCnt'),
        'objInfrstrTrashAreaCnt' : object_data.get('objInfrstrTrashAreaCnt'),
        'objLkClassDesc' : object_data.get('objLkClassDesc'),
        'objLkFinishTypeDesc' : object_data.get('objLkFinishTypeDesc'),
        'objLkFreePlanDesc' : object_data.get('objLkFreePlanDesc'),
        'objPriceAvg' : object_data.get('objPriceAvg'),
        'objReady100PercDt' : object_data.get('objReady100PercDt'),
        'objSquareLiving' : object_data.get('objSquareLiving'),
        'objStatus' : object_data.get('objStatus'),
        'soldOutPerc' : object_data.get('soldOutPerc'),
        'wallMaterialShortDesc' : object_data.get('wallMaterialShortDesc'),
        } , index=[iter_obj])])
    for iter_foto in range(len(object_data.get('photoRenderDTO'))):
        df_objects_photo = pd.concat([df_objects_photo, pd.DataFrame({'foto': object_data.get('photoRenderDTO')[iter_foto].get('objRenderPhotoUrl')}, index=[object_data.get('id')])])


100%|██████████| 1071/1071 [00:09<00:00, 107.12it/s]


In [18]:
df_objects_data.to_excel('objects_data.xlsx')
df_objects_data.to_pickle('objects_data.pkl')
df_objects_data.to_csv('objects_data.csv', sep=";")
sql_db = sql.connect('sql_objects_data.db')
sql_con = sql_db.cursor()
df_objects_data.to_sql('objects', sql_db, if_exists='replace', index=False)
sql_con.close()

In [19]:
df_objects_photo = (df_objects_photo.reset_index()).rename(columns={'index': 'id'})

In [21]:
dir_name = ''
os.mkdir('PhotoObjects')
os.chdir('PhotoObjects')
# for i in tqdm.trange(len(df_objects_photo)):
for i in tqdm.trange(50):
    if dir_name != str(df_objects_photo.iloc[i]['id']):
        dir_name = str(df_objects_photo.iloc[i]['id'])
        os.mkdir(dir_name)
    with open(os.getcwd() + '\\' + dir_name + '\\' + str(i) + '.jpg', 'wb') as f:
        f.write((rq.get(df_objects_photo.iloc[i]['foto'])).content)
        f.close
    time.sleep(1/4)
os.chdir(os.path.pardir)
os.chdir(os.path.pardir)
os.getcwd()

100%|██████████| 50/50 [00:29<00:00,  1.67it/s]


'C:\\Users\\16706093\\PycharmProjects\\School_DS'